In [ ]:
import numpy as np
from numpy import linalg
from numba import jit
import math
import matplotlib as mpl
from matplotlib import cm
import matplotlib.pyplot as plt
import matplotlib.colors as colors
import matplotlib.transforms as mtransforms
from matplotlib.ticker import (MultipleLocator, AutoMinorLocator)

from scipy.constants import pi
from scipy.interpolate import RectBivariateSpline, griddata

import os
import ast
import pandas as pd

In [ ]:
def set_pandas_display_options() -> None:
    """Set pandas display options."""
    # Ref: https://stackoverflow.com/a/52432757/
    display = pd.options.display

    display.max_columns = None
    display.max_rows = None
    display.max_colwidth = None
    display.width = None
    display.chop_threshold = None
    display.precision = None  # set as needed
    display.float_format = '{:,.15f}'.format

set_pandas_display_options()

## Chern coefficients

In [ ]:
def n_u1t1(theta,Ch_u1t1,Ch_u1u2t1t2):
    return Ch_u1t1 + theta * Ch_u1u2t1t2

def n_u2t2(theta,Ch_u2t2,Ch_u1u2t1t2):
    return Ch_u2t2 + theta * Ch_u1u2t1t2

def n_(theta,Ch_,Ch_u1t1,Ch_u2t2,Ch_u1u2t1t2):
    return Ch_ - theta * Ch_u1t1 - theta * Ch_u2t2 - theta**2 * Ch_u1u2t1t2

In [ ]:
files = [x[0] for x in os.walk('./data/')][1::]

df = pd.DataFrame()
for file in files:
    
    data = pd.read_json(file+'/params.json',orient='index').transpose()
    data["key"] = file[7:]
    
    Chern_numbers = np.load(file+"/chern_numbers_tau1tau2u1u2.npy",allow_pickle=True)[()]

    for label in Chern_numbers:
        if label=="":
            data["IDS"]=Chern_numbers[label].real
        else:
            data[label]=Chern_numbers[label].real
    df = pd.concat([df,data])

display(df.set_index('key').sort_index(ascending=False))

In [ ]:
keys = [
'1702039394',
'1702039397',
'1702039403',
'1702039419',
'1702039482',
'1702039610',
'1702039838',
'1702040263',
'1702041491',
'1702044390',
'1702048724',
'1702058077',
'1702070968',
]

In [ ]:
thetas = np.zeros(len(keys))
sizes = np.zeros(len(keys))

ChernNumbers = np.zeros((8,len(keys)))

# New Convention: tau1 = -t1 and order u1u2t1t2
# u1u2         -> u1u2
# tau1u1       -> (-1)^2 u1t1
# tau2u1       -> (-1)^2 u1t2
# tau1u2       -> (-1)^2 u2t1
# tau2u2       -> (-1)^2 u2t2
# tau1tau2     -> (-1)^2 t1t2
# tau1tau2u1u2 -> (-1)^2 u1u2t1t2

for i,key in enumerate(keys):
    thetas[i] = df.set_index('key').at[key,'q']
    sizes[i] = df.set_index('key').at[key,'system_sizes'][0]
    ChernNumbers[0,i] = df.set_index('key').at[key,'IDS']          # _
    ChernNumbers[1,i] = df.set_index('key').at[key,'u1u2']         # u1u2
    ChernNumbers[2,i] = df.set_index('key').at[key,'tau1u1']       # u1t1
    ChernNumbers[3,i] = df.set_index('key').at[key,'tau2u1']       # u1t2
    ChernNumbers[4,i] = df.set_index('key').at[key,'tau1u2']       # u2t1
    ChernNumbers[5,i] = df.set_index('key').at[key,'tau2u2']       # u2t2
    ChernNumbers[6,i] = df.set_index('key').at[key,'tau1tau2']     # t1t2
    ChernNumbers[7,i] = df.set_index('key').at[key,'tau1tau2u1u2'] # u1u2t1t2

ChernCoefficients = np.zeros((8,len(keys)))

ChernCoefficients[0] = n_(thetas,ChernNumbers[0],ChernNumbers[2],ChernNumbers[5],ChernNumbers[7]) # _
ChernCoefficients[1] = ChernNumbers[1]                                                            # u1u2
ChernCoefficients[2] = n_u1t1(thetas,ChernNumbers[2],ChernNumbers[7])                             # u1t1
ChernCoefficients[3] = ChernNumbers[3]                                                            # u1t2
ChernCoefficients[4] = ChernNumbers[4]                                                            # u2t1
ChernCoefficients[5] = n_u2t2(thetas,ChernNumbers[5],ChernNumbers[7])                             # u2t2
ChernCoefficients[6] = ChernNumbers[6]                                                            # t1t2
ChernCoefficients[7] = ChernNumbers[7]                                                            # u1u2t1t2 

In [ ]:
CN = {r'$Ch_J$': [r'$Ch_\emptyset$',r'$Ch_{u_1u_2}$',r'$Ch_{u_1t_1}$','$Ch_{u_1t_2}$',r'$Ch_{u_2t_1}$',r'$Ch_{u_2t_2}$',r'$Ch_{t_1t_2}$',r'$Ch_{u_1u_2t_1t_2}$']}

for i in range(len(keys)):
    CN[r'Size %g'%(sizes[i])] = ChernNumbers[:,i]

CN = pd.DataFrame(CN)

CC = {r'$n_J$' : [r'$n_\emptyset$',r'$n_{u_1u_2}$',r'$n_{u_1t_1}$',r'$n_{u_1t_2}$',r'$n_{u_2t_1}$',r'$n_{u_2t_2}$',r'$n_{t_1t_2}$',r'$n_{u_1u_2t_1t_2}$']}

for i in range(len(keys)):
    CC[r'Size %g'%(sizes[i])] = ChernCoefficients[:,i]

CC = pd.DataFrame(CC)

display(CN,CC)

## Plot

In [ ]:
set_pandas_display_options()
plt.rcParams['figure.figsize'] = [8, 6]
plt.rcParams['savefig.facecolor'] = "white"
mpl.rcParams['figure.dpi'] = 300
mpl.rcParams['axes.linewidth'] = 1.2
mpl.rcParams['text.usetex'] = True
mpl.rcParams['mathtext.fontset'] = 'cm'
mpl.rcParams['font.family'] = 'cmu serif'
tfs    = 30 #title font size
lfs    = 24 #label font size
fs     = 18 #font size
cbarfs = 20 #colorbar font size

In [ ]:
fig, ax = plt.subplots()
ax.semilogy(sizes,np.abs(ChernNumbers[6]-1),label=r'${t_1t_2}$',c='Darkblue')
ax.semilogy(sizes,np.abs(ChernNumbers[1]-1),label=r'${u_1u_2}$',c='C1')
ax.semilogy(sizes,np.abs(ChernNumbers[7]+1),label=r'${u_1u_2t_1t_2}$',c='C3')

ax.set_xlabel("\n N\n"+r"$\vartheta$",fontsize=lfs)
ax.set_ylabel(r"$\Delta Ch_I$",fontsize=lfs)

ax.tick_params(axis='y', which='major', labelsize=lfs)
ax.tick_params(axis='x', which='major', labelsize=fs)
ax.set_xticks(sizes)
ax.set_yticks([10**-i for i in range(1,12,2)])
ax.set_ylim(10**-12,1)

theta_labels = ['\n{:.4f}'.format(t) if i%2==0 else '\n\n{:.4f}'.format(t) for i,t in enumerate(thetas)]
sax = ax.secondary_xaxis(location=0)
sax.set_xticks(sizes)
sax.set_xticklabels(theta_labels)
sax.tick_params(axis='x', which='major', labelsize=14)

ax.legend(fontsize=fs)

plt.tight_layout()
plt.savefig("./plots/Error_estimate_size_delta-5", dpi=300, bbox_inches = 'tight')